<a href="https://colab.research.google.com/github/srilav/machinelearning/blob/main/M3_SNB_MiniProject_1_LinearRegression_Regularization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Certification Program in Computational Data Science
## A program by IISc and TalentSprint
### Mini-Project: Linear Regression with Regularization

**DISCLAIMER:** THIS NOTEBOOK IS PROVIDED ONLY AS A REFERENCE SOLUTION NOTEBOOK FOR THE MINI-PROJECT. THERE MAY BE OTHER POSSIBLE APPROACHES/METHODS TO ACHIEVE THE SAME RESULTS.

## Problem Statement

Predict the bike-sharing counts per hour based on features including weather, day, time, humidity, wind speed, season e.t.c.

## Learning Objectives

At the end of the mini-project, you will be able to :

* perform data exploration and visualization
* implement linear regression using sklearn and optimization
* apply regularization on regression using lasso and ridge techniques
* calculate and compare the MSE value of each regression technique
* analyze the features that are best contributing to the target

### Dataset

The dataset chosen for this mini-project is [Bike Sharing Dataset](https://archive.ics.uci.edu/ml/datasets/bike+sharing+dataset).  This dataset contains the hourly and daily count of rental bikes between the years 2011 and 2012 in the capital bike share system with the corresponding weather and seasonal information. This dataset consists of 17389 instances of each 16 features. 

Bike sharing systems are a new generation of traditional bike rentals where the whole process from membership, rental and return has become automatic. Through these systems, the user can easily rent a bike from a particular position and return to another position. Currently, there are about over 500 bike-sharing programs around the world which is composed of over 500 thousand bicycles. Today, there exists great interest in these systems due to their important role in traffic, environmental and health issues.

Apart from interesting real world applications of bike sharing systems, the characteristics of data being generated by these systems make them attractive for the research. As opposed to other transport services such as bus or subway, the duration of travel, departure and arrival position are explicitly recorded in these systems. This feature turns bike sharing system into a virtual sensor network that can be used for sensing mobility in the city. Hence, it is expected that the most important events in the city could be detected via monitoring these data.

<img src="https://s26551.pcdn.co/wp-content/uploads/2012/02/resize-va-sq-bikeshare.jpg" alt="drawing" width="400"/>

### Data Fields

* dteday - hourly date
* season - 1 = spring, 2 = summer, 3 = fall, 4 = winter
* hr - hour
* holiday - whether the day is considered a holiday
* workingday - whether the day is neither a weekend nor holiday
* weathersit -<br>
    1 - Clear, Few clouds, Partly cloudy, Partly cloudy <br>
    2 - Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist<br>
    3 - Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds<br>
    4 - Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog<br>   
* temp - temperature in Celsius
* atemp - "feels like" temperature in Celsius
* humidity - relative humidity
* windspeed - wind speed
* casual - number of non-registered user rentals initiated
* registered - number of registered user rentals initiated
* cnt - number of total rentals

## Information

**Regularization:** It is a form of regression, that constrains or regularizes or shrinks the coefficient estimates towards zero. In other words, this technique discourages learning a more complex or flexible model, to avoid the risk of overfitting. A simple relation for linear regression looks like this.

$Y ≈ β_0 + β_1 X_1 + β_2 X_2 + …+ β_p X_p$

 Here $Y$ represents the learned relation and $β$ represents the coefficient estimates for different variables or predictors(X).
 
 If there is noise in the training data, then the estimated coefficients won’t generalize well to the future data. This is where regularization comes in and shrinks or regularizes these learned estimates towards zero.
 
Below are the Regularization techniques:
 
 * Ridge Regression
 * Lasso Regression
 * Elasticnet Regression

## Grading = 10 Points

### Download the dataset

In [ ]:
!wget -qq https://cdn.iisc.talentsprint.com/CDS/MiniProjects/Bike_Sharing_Dataset.zip
!unzip -qq Bike_Sharing_Dataset.zip

#### Importing Necessary Packages

In [ ]:
# Loading the Required Packages
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.metrics import r2_score

### Data Loading

In [ ]:
# Reading Our Dataset
bikeshare = pd.read_csv('hour.csv')

first five rows of dataset

In [ ]:
bikeshare.head()

In [ ]:
bikeshare.dtypes

### Task flow with respect to feature processing and model training

* Identify continuous features

* Identify categorical features

* Apply scaling on continuous features 

* Apply one-hot encoding on categorical features

* Create features by concatenating all one hot encoded features and scaled features except target variables

* Find the coefficients of the features using normal equation and find the cost (error)

* Apply batch gradient descent technique by taking one target variable (cnt) and find the best coefficients

* Apply SGD Regressor using sklearn

* Apply linear regression using sklearn by taking two target variables (casual, registered)

* Apply Lasso, Ridge, Elasticnet Regression

### EDA &  Visualization ( 2 points)

#### Visualize the hour (hr) column with an appropriate plot and find the busy hours of bike sharing

In [ ]:
bikeshare.groupby('hr').sum('cnt')['cnt'].plot.bar()

#### Visualize the distribution of count, casual and registered variables

In [ ]:
# distribution of casual
sns.distplot(bikeshare.casual);

In [ ]:
# distribution of registered
sns.distplot(bikeshare.registered);

In [ ]:
# distribution of count
sns.distplot(bikeshare.cnt);

#### Describe the relation of weekday, holiday and working day

In [ ]:
# Working days from 1-5 (mon-fri)
bikeshare[bikeshare.workingday==1].weekday.unique()

In [ ]:
# Holiday possible on working days
bikeshare[bikeshare.holiday==1].weekday.unique()

In [ ]:
# Not a holiday, not a working day (Sun, Sat)
bikeshare[(bikeshare.holiday==0) & (bikeshare.workingday==0)].weekday.unique()

#### Visualize the monthly wise count of both casual and registered for the year 2011 and 2012 separately.

Hint: Stacked barchart

In [ ]:
# stacked bar chart for year 2011
bikeshare[bikeshare.yr==0].groupby('mnth').sum(['casual','registered'])[['casual','registered']].plot.bar(stacked=True);
plt.title("Casual and Registered in 2011")
plt.show()

In [ ]:
# stacked bar chart for year 2012
bikeshare[bikeshare.yr==1].groupby('mnth').sum(['casual','registered'])[['casual','registered']].plot.bar(stacked=True)
plt.title("Casual and Registered in 2012")
plt.show()

#### Analyze the correlation between features with heatmap

In [ ]:
sns.heatmap(bikeshare.iloc[:,:].corr(), cmap='RdBu')

#### Visualize the box plot of casual and registered variables to check the outliers

In [ ]:
fig, axes = plt.subplots(nrows=1,ncols=2)
sns.boxplot(data=bikeshare,y="casual",orient="v",ax=axes[0])
sns.boxplot(data=bikeshare,y="registered",orient="v",ax=axes[1])
plt.show()

### Pre-processing and Data Engineering (1 point)

#### Drop unwanted columns

In [ ]:
bikeshare1 = bikeshare.drop(['instant', 'dteday'], axis = 1)
bikeshare1.shape

#### Identify categorical and continuous variables


In [ ]:
# Identifying categorical and continuous variables
cont_features = ['temp','atemp','hum','windspeed'] #,'casual','registered','cnt']
categorical_features = ['season', 'yr', 'mnth','hr','holiday','weekday','weathersit']

In [ ]:
target = bikeshare1[['casual','registered','cnt']]
features = bikeshare1.drop(target.columns,axis=1)
target.shape, features.shape

#### Select the features and target and split the dataset

As there are 3 target variables, choose the count (`cnt`) variable.

In [ ]:
target1 = target[['cnt']]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(features, target1,test_size=0.1)
x_train.shape, y_train.shape, x_test.shape

#### Feature scaling

Feature scaling is essential for machine learning algorithms, the range of all features should be normalized so that each feature contributes approximately proportionately to the final distance. Apply scaling on the continuous variables on the given data.

Hint: `MinMaxScaler` or `StandardScaler`



In [ ]:
import warnings
warnings.filterwarnings("ignore")
std_scaler = StandardScaler()
x_train[cont_features] = std_scaler.fit_transform(x_train[cont_features])
x_test[cont_features] = std_scaler.fit_transform(x_test[cont_features])

#### Apply one-hot encode on the categorical data

One-hot encoding is applied on the categorical variables, which should not have a different weight or order attached to them, it is presumed that all categorical variables have equivalent "values". This means that you cannot simply order them from zero to the number of categories as this would imply that the earlier categories have less "value" than later categories.

Hint: `sklearn.preprocessing.OneHotEncoder`

In [ ]:
onehot = OneHotEncoder()
onehot_encoder = onehot.fit(x_train[categorical_features])
onehot_encoded_train = onehot_encoder.transform(x_train[categorical_features]).toarray()
onehot_encoded_test = onehot_encoder.transform(x_test[categorical_features]).toarray()

In [ ]:
onehot_encoded_train.shape

In [ ]:
x_train_final = np.concatenate((x_train[cont_features].values, onehot_encoded_train), axis=1)
x_test_final = np.concatenate((x_test[cont_features].values, onehot_encoded_test), axis=1)
x_train_final.shape

#### Specify features and targets after applying scaling and one-hot encoding

In [ ]:
x_train = x_train_final
x_test = x_test_final
x_train_final.shape, y_train.shape, x_test.shape, y_test.shape

### Implement the linear regression by finding the coefficients using below approaches (3 points)

* Find the coefficients using Normal equation and find the error

* Implement batch gradient descent

* SGD Regressor from sklearn

#### Implementation using Normal Equation

$\theta = (X^T X)^{-1} . (X^T Y)$

$θ$ is the hypothesis parameter that defines the coefficients

$X$ is the input feature value of each instance

$Y$ is Output value of each instance

To solve the normal equation compute least-squares solution by using `scipy.linalg`

Hint: [scipy.linalg.lstsq](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.lstsq.html)

In [ ]:
y = y_train.values
# Adding ones to X
X = np.append(np.ones((x_train.shape[0],1)),x_train, axis=1)
X.shape

In [ ]:
# X_transpose * X
X_t = np.transpose(X)
X_Xt_dot = X_t.dot(X)

# inverse of (X * X_transpose)
temp1 = np.linalg.inv(X_Xt_dot)
temp1.shape

In [ ]:
# X_transpose * Y
temp2 = X_t.dot(y)
# Inverse of (X_transpose * X) * (X_transpose * Y)
coefs = temp1.dot(temp2)
coefs.shape

In [ ]:
# Above steps in one line
y = y_train.values
X_b = np.concatenate((np.ones((x_train.shape[0], 1)), x_train),axis=1)
theta_star = np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y)
theta_star.shape

In [ ]:
ypredict = X_b.dot(theta_star.ravel())
mean_squared_error(ypredict, y_train.values)

In [ ]:
def Calc_MSE(X, y_test, coefficients):

    X = np.append(np.ones((X.shape[0],1)),X, axis=1)
    score = mean_squared_error(y_test.values, X.dot(coefficients))
    return score

In [ ]:
# objective(X,y, coefs)
train_error = Calc_MSE(x_train, y_train, coefs)
test_error = Calc_MSE(x_test, y_test, coefs)
train_error, test_error

In [ ]:
from scipy.linalg import lstsq

x_train, x_test, y_train, y_test = train_test_split(features, target1)
y = y_train.values
X = np.append(np.ones((x_train.shape[0],1)),x_train, axis=1)
p, res, rnk, s = lstsq(X, y)
print(np.sum((X.dot(p)-y)**2)/X.shape[0])

#### Implementing Linear regression using batch gradient descent

Initialize the random coefficients and optimize the coefficients in the iterative process by calculating cost and finding the gradient.

Hint: [link](https://medium.com/@lope.ai/multivariate-linear-regression-from-scratch-in-python-5c4f219be6a)

In [ ]:
X = x_train
y = y_train
# Adding ones to X
X = np.append(np.ones((X.shape[0],1)),X, axis=1)
X.shape

In [ ]:
def cost_function(X, Y, B):
  return mean_squared_error(Y, X.dot(B))

In [ ]:
def batch_gradient_descent(X, Y, B, alpha, iterations):
  cost_history = [0] * iterations
  m = len(Y)
  for iteration in range(iterations):
    #print(iteration)
    h = X.dot(B)
    loss = h - Y #change the variable name
    gradient = X.T.dot(loss) / m
    B = B - alpha * gradient
    cost = cost_function(X, Y, B)
    cost_history[iteration] = cost
  return B, cost_history

B = np.random.randn(X.shape[1])
alpha = 0.005
iter_ = 50000
newB, cost_history = batch_gradient_descent(X, y.values.ravel(), B, alpha, iter_)
newB, cost_history[-1]

In [ ]:
# test error
X_test = np.append(np.ones((x_test.shape[0],1)),x_test, axis=1)
cost_function(X_test,y_test, newB)

#### SGD Regressor

Use the SGD regressor from sklearn with one target variable and find the error

Hint: [SGDRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html)

In [ ]:
sgd = linear_model.SGDRegressor()
sgd = sgd.fit(x_train, y_train)
print("score is ",sgd.score(x_test, y_test))
mean_squared_error(sgd.predict(x_test), y_test )

### Linear regression using sklearn (2 points)

Implement the linear regression model using sklearn

* Import Linear Regression and fit the train data

* Predict the test data and find the error

Hint: [LinearRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)

#### Select the features  and split the data into train and test

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(features, target1)
xtrain.shape, ytrain.shape

In [ ]:
regr_linear = linear_model.LinearRegression()
regr_linear.fit(xtrain, ytrain)
predicted = regr_linear.predict(xtest)

#### Calculate the mean squared error of the actual and predicted data

In [ ]:
mse_linear = mean_squared_error(ytest, predicted, multioutput = 'uniform_average')
mse_linear

#### Calculate the $R^2$ (coefficient of determination) of the actual and predicted data

In [ ]:
r2_score(ytrain, regr_linear.predict(xtrain)), r2_score(ytest, predicted)

#### summarize the importance of features and create a bar chart

Prediction is the weighted sum of the input values e.g. linear regression, and extensions that add regularization, such as ridge regression and the elastic net find a set of coefficients to use in the weighted sum to make a prediction. These coefficients can be used directly as a crude type of feature importance score.

This assumes that the input variables have the same scale or have been scaled prior to fitting a model.

In [ ]:
# coefficients of casual variable
importance_casual = regr_linear.coef_[0,]
plt.bar([x for x in range(len(importance_casual))], importance_casual)
plt.xticks(range(59))
plt.show()

In [ ]:
# coefficients of registered variable
# importance_registered = regr_linear.coef_[1,]
# plt.bar([x for x in range(len(importance_registered))], importance_registered)
# plt.xticks(range(59))
# plt.show()

### Regularization methods (2 points)

#### Apply lasso regression

* Apply Lasso regression with different alpha values given below and find the best alpha that gives the least error.
* Calculate the metrics for the actual and predicted

Hint: [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html)

In [ ]:
# setting up alpha
alpha = [0.0001, 0.001,0.01, 0.1, 1, 10, 100]

In [ ]:
for a in alpha:
    regr_lasso = linear_model.Lasso(alpha = a)
    regr_lasso.fit(xtrain, ytrain)
    mse_lasso_sk = mean_squared_error(ytest, regr_lasso.predict(xtest), multioutput = 'uniform_average')
    print(a, "=====",mse_lasso_sk) 

In [ ]:
# with best alpha chosen from above
regr_lasso = linear_model.Lasso(alpha = 0.0001)
regr_lasso.fit(xtrain, ytrain)
mse_lasso_sk = mean_squared_error(ytest, regr_lasso.predict(xtest), multioutput = 'uniform_average')
print("Lasso MSE:",mse_lasso_sk)
print("Lasso r2_score",r2_score(ytrain, regr_lasso.predict(xtrain)))

#### Apply ridge regression

* Apply Lasso regression with different alpha values given and find the best alpha that gives the least error.
* Calculate the metrics for the actual and predicted

Hint: [Ridge](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html)

In [ ]:
for a in alpha:
    regr_ridge = linear_model.Ridge(alpha = a)
    regr_ridge.fit(xtrain, ytrain)
    mse_ridge_sk = mean_squared_error(ytest, regr_ridge.predict(xtest), multioutput = 'uniform_average')
    print(a, "=====",mse_ridge_sk)

In [ ]:
# with best alpha chosen from above
regr_ridge = linear_model.Ridge(alpha = 0.001)
regr_ridge.fit(xtrain, ytrain)
mse_ridge_sk = mean_squared_error(ytest, regr_ridge.predict(xtest), multioutput = 'uniform_average')
print("Ridge MSE:",mse_ridge_sk)
print("Ridge r2_score:",r2_score(ytrain, regr_ridge.predict(xtrain)))

#### Apply elasticnet regression

* Apply elasticnet regression with different alpha values given and find the best alpha that gives the least error.
* Calculate the metrics for the actual and predicted

Hint: [ElasticNet](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html)

In [ ]:
for a in alpha:
    elasticnet_regr = linear_model.ElasticNet(alpha=a)
    elasticnet_regr.fit(xtrain, ytrain)
    mse_elatic_sk = mean_squared_error(ytest, elasticnet_regr.predict(xtest), multioutput = 'uniform_average')
    print(a,"====",mse_elatic_sk)

In [ ]:
# Elasticnet
elasticnet_regr = linear_model.ElasticNet(alpha=0.01)
elasticnet_regr.fit(xtrain, ytrain)
mse_elatic_sk = mean_squared_error(ytest, elasticnet_regr.predict(xtest), multioutput = 'uniform_average')
print("Elasticnet MSE:",mse_elatic_sk)
print("Elasticnet r2_score",r2_score(ytrain, elasticnet_regr.predict(xtrain)))

### Determine if there is a reduction in error if two target variables are considered

Consider (`Casual, Registered`) as target and find the error by implementing Linear Regression model from sklearn

#### Solution of Linear regression with 2 variables

In [ ]:
target2 = bikeshare1[['casual','registered']]

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(features, target2)
xtrain.shape, ytrain.shape

In [ ]:
regr_linear = linear_model.LinearRegression()
regr_linear.fit(xtrain, ytrain)
predicted = regr_linear.predict(xtest)

In [ ]:
mse_linear = mean_squared_error(ytest, predicted, multioutput = 'uniform_average')
mse_linear

In [ ]:
r2_score(ytrain, regr_linear.predict(xtrain)), r2_score(ytest, predicted)

### Report Analysis

* Describe your interpretation of the methods that are used to implement linear regression covered in this mini project.
* Comment on performance of the algorithms/methods used.
* Comment about the nature of the data and fitment of linear regression for this data.
* Can you perform a non linear curve fitting using linear regression? If yes, How?
